# Data Flows with Kafka

Components:
- Kafka for persistence and information exchange.
- Proxy scheduler for ease of testing.
- Kotlin transformer engine to wrap Kafka Streams

## Prerequisites

- Docker
- JRE (and JDK for development)
- `kafkacat`/`kcat` or other CLI tool for interacting with Kafka
    - https://docs.confluent.io/platform/current/app-development/kafkacat-usage.html
- _(optional)_ `grpcurl` for checking interaction with Kotlin engine

## Setup

You will need to run Kafka and a (proxy) scheduler before the Kotlin engine can start up successfully.

```bash
# .../scheduler
make start-kafka-host
make start-proxy-local
```

```bash
# .../data-flow
./gradlew build
```

The input topic for a particular step for the data-flow engine is the _output_ of a model/component, therefore contains _responses_.

Conversely, the data-flow engine should be writing _requests_ to the _input_ topic for another model/compoment, but this is the _output_ of that step.

So, below we denote a step by number, e.g. 1 or 2, and talk about the inputs and outputs for that step for variable names.
The actual topic names are the other way around.

In [1]:
inputTopic1 = 'seldon.some-namespace.some-model-1.outputs'
outputTopic1 = 'seldon.some-namespace.some-model-2.inputs'

inputTopic2 = 'seldon.some-namespace.some-model-3.outputs'
outputTopic2 = 'seldon.some-namespace.some-model-4.inputs'

## Check there are no existing topics (no cheating)

In [2]:
!kafkacat -b localhost:9092 -L | grep -i seldon -A1

## Write to Kafka topic

In [3]:
!make -C .. build-dataflow-producer

make: Entering directory '/home/agr/source/agrski/seldon-core-v2/scheduler'
go build -o data-flow/scripts/bin/producer ./data-flow/scripts/producer.go ./data-flow/scripts/common.go
make: Leaving directory '/home/agr/source/agrski/seldon-core-v2/scheduler'


In [4]:
!../data-flow/scripts/bin/producer --input-topics $inputTopic1,$inputTopic2

In [5]:
!kafkacat -b localhost:9092 -L | grep -i seldon -A1

  topic "seldon.some-namespace.some-model-1.outputs" with 1 partitions:
    partition 0, leader 1, replicas: 1, isrs: 1
  topic "seldon.some-namespace.some-model-3.outputs" with 1 partitions:
    partition 0, leader 1, replicas: 1, isrs: 1


## Consume from Kafka topic

Half the messages here should be ignored by the data-flow engine due to having headers that do not match the pipeline name, i.e. 10 messages here -> 5 messages on output topics.

In [6]:
!kafkacat -b localhost:9092 -t $inputTopic1 -C -o beginning -e -f 'Offset %o\tlength %S\tcontents %s\n' -s value=i

Offset 0	length 77	contents 168455535
Offset 1	length 77	contents 168455535
Offset 2	length 77	contents 168455535
Offset 3	length 77	contents 168455535
Offset 4	length 77	contents 168455535
Offset 5	length 77	contents 168455535
Offset 6	length 77	contents 168455535
Offset 7	length 77	contents 168455535
Offset 8	length 77	contents 168455535
Offset 9	length 77	contents 168455535
% Reached end of topic seldon.some-namespace.some-model-1.outputs [0] at offset 10: exiting


In [7]:
!kafkacat -b localhost:9092 -t $inputTopic2 -C -o beginning -e -f 'Offset %o\tlength %S\tcontents %s\n' -s value=i

Offset 0	length 77	contents 168455535
Offset 1	length 77	contents 168455535
Offset 2	length 77	contents 168455535
Offset 3	length 77	contents 168455535
Offset 4	length 77	contents 168455535
Offset 5	length 77	contents 168455535
Offset 6	length 77	contents 168455535
Offset 7	length 77	contents 168455535
Offset 8	length 77	contents 168455535
Offset 9	length 77	contents 168455535
% Reached end of topic seldon.some-namespace.some-model-3.outputs [0] at offset 10: exiting


## Transform data

In [ ]:
! pushd ../data-flow/ \
    && ./gradlew run \
        --no-daemon \
        --args="--kafka-bootstrap-servers=localhost:9092 --upstream-host=localhost --upstream-port=10101 --num-cores=2" \
    && popd

In [8]:
!kafkacat -b localhost:9092 -L | grep -i seldon -A1

  topic "seldon.some-namespace.some-model-1.outputs" with 1 partitions:
    partition 0, leader 1, replicas: 1, isrs: 1
  topic "seldon.some-namespace.some-model-2.inputs" with 1 partitions:
    partition 0, leader 1, replicas: 1, isrs: 1
  topic "seldon.some-namespace.some-model-3.outputs" with 1 partitions:
    partition 0, leader 1, replicas: 1, isrs: 1
  topic "seldon.some-namespace.some-model-4.inputs" with 1 partitions:
    partition 0, leader 1, replicas: 1, isrs: 1


In [9]:
!kafkacat -b localhost:9092 -t $outputTopic1 -C -o beginning -e -f 'Offset %o\tlength %S\tcontents %s\n' -s value=i

Offset 0	length 62	contents 436483123
Offset 1	length 62	contents 436483123
Offset 2	length 62	contents 436483123
Offset 3	length 62	contents 436483123
Offset 4	length 62	contents 436483123
% Reached end of topic seldon.some-namespace.some-model-2.inputs [0] at offset 5: exiting


In [10]:
!kafkacat -b localhost:9092 -t $outputTopic2 -C -o beginning -e -f 'Offset %o\tlength %S\tcontents %s\n' -s value=i

Offset 0	length 34	contents 436483123
Offset 1	length 34	contents 436483123
Offset 2	length 34	contents 436483123
Offset 3	length 34	contents 436483123
Offset 4	length 34	contents 436483123
% Reached end of topic seldon.some-namespace.some-model-4.inputs [0] at offset 5: exiting


In [12]:
!make -C .. build-dataflow-consumer

make: Entering directory '/home/agr/source/agrski/seldon-core-v2/scheduler'
go build -o data-flow/scripts/bin/consumer ./data-flow/scripts/consumer.go ./data-flow/scripts/common.go
make: Leaving directory '/home/agr/source/agrski/seldon-core-v2/scheduler'


In [13]:
!../data-flow/scripts/bin/consumer --output-topics $outputTopic1,$outputTopic2

INFO[0003] received inference response on topic seldon.some-namespace.some-model-4.inputs: id:"4312" inputs:{name:"tensor1" datatype:"INT32" shape:1 shape:2 contents:{int_contents:0 int_contents:0}} 
INFO[0003] received inference response on topic seldon.some-namespace.some-model-4.inputs: id:"4312" inputs:{name:"tensor1" datatype:"INT32" shape:1 shape:2 contents:{int_contents:1 int_contents:1}} 
INFO[0003] received inference response on topic seldon.some-namespace.some-model-4.inputs: id:"4312" inputs:{name:"tensor1" datatype:"INT32" shape:1 shape:2 contents:{int_contents:2 int_contents:2}} 
INFO[0003] received inference response on topic seldon.some-namespace.some-model-4.inputs: id:"4312" inputs:{name:"tensor1" datatype:"INT32" shape:1 shape:2 contents:{int_contents:3 int_contents:3}} 
INFO[0003] received inference response on topic seldon.some-namespace.some-model-4.inputs: id:"4312" inputs:{name:"tensor1" datatype:"INT32" shape:1 shape:2 contents:{int_contents:4 int_contents:4}} 
